<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/DEEPCTR01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U deepctr-torch

In [ ]:
!wget https://raw.githubusercontent.com/shenweichen/DeepCTR/refs/heads/master/examples/criteo_sample.txt -O ./criteo_sample.txt
!head -5 ./criteo_sample.txt

--2025-02-19 08:38:09--  https://raw.githubusercontent.com/shenweichen/DeepCTR/refs/heads/master/examples/criteo_sample.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 52518 (51K) [text/plain]
Saving to: ‘./criteo_sample.txt’

./criteo_sample.txt 100%[===================>]  51.29K  --.-KB/s    in 0.05s   

2025-02-19 08:38:09 (1.05 MB/s) - ‘./criteo_sample.txt’ saved [52518/52518]

label,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,C15,C16,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,,3,260.0,,17668.0,,,33.0,,,,0.0,,05db9164,08d6d899,9143c832,f56b7dd5,25c83c98,7e0ccccf,df5c2d18,0b153874,a73ee510,8f48ce11,a7b606c4,ae1bb660,eae197fd,b28479f6,bfef54b3,bad5ee18,e5ba7672,87c6f83c,,,0429f84b,,3a171ecb,c0d61a5c,,
0,,-1,

In [ ]:
#movielens_sample.txt

!wget https://raw.githubusercontent.com/shenweichen/DeepCTR/refs/heads/master/examples/movielens_sample.txt -O ./movielens_sample.txt
!head -5 ./movielens_sample.txt

--2025-02-19 08:38:09--  https://raw.githubusercontent.com/shenweichen/DeepCTR/refs/heads/master/examples/movielens_sample.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15155 (15K) [text/plain]
Saving to: ‘./movielens_sample.txt’

./movielens_sample. 100%[===================>]  14.80K  --.-KB/s    in 0.003s  

2025-02-19 08:38:09 (5.26 MB/s) - ‘./movielens_sample.txt’ saved [15155/15155]

user_id,movie_id,rating,timestamp,title,genres,gender,age,occupation,zip
3299,235,4,968035345,Ed Wood (1994),Comedy|Drama,F,25,4,19119
3630,3256,3,966536874,Patriot Games (1992),Action|Thriller,M,18,4,77005
517,105,4,976203603,"Bridges of Madison County, The (1995)",Drama|Romance,F,25,14,55408
785,2115,3,975430389,Indiana Jones and the Temple of Doom (1984),Action|Adv

In [ ]:
#byterec_sample.txt

!wget https://raw.githubusercontent.com/shenweichen/DeepCTR-Torch/refs/heads/master/examples/byterec_sample.txt -O ./byterec_sample.txt
!head -5 ./byterec_sample.txt

--2025-02-19 08:38:10--  https://raw.githubusercontent.com/shenweichen/DeepCTR-Torch/refs/heads/master/examples/byterec_sample.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11048 (11K) [text/plain]
Saving to: ‘./byterec_sample.txt’

./byterec_sample.tx 100%[===================>]  10.79K  --.-KB/s    in 0.001s  

2025-02-19 08:38:10 (8.22 MB/s) - ‘./byterec_sample.txt’ saved [11048/11048]

37448	115	567569	44888	42	0	0	0	1699	43981	53085738314	9
8623	82	1209192	10098	106	0	1	0	-1	11996	53086444998	8
9629	31	1209193	184752	109	0	1	0	-1	32093	53085591140	5
52799	175	1209194	109629	101	0	1	0	-1	33106	53085915481	6
38008	-1	1209195	456237	11	1	0	1	56	18558	53085805030	9


In [ ]:
import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names

data = pd.read_csv('./criteo_sample.txt')

sparse_features = ['C' + str(i) for i in range(1, 27)]
dense_features = ['I' + str(i) for i in range(1, 14)]

data[sparse_features] = data[sparse_features].fillna('-1', )
data[dense_features] = data[dense_features].fillna(0, )
target = ['label']

In [ ]:
for feat in sparse_features:
    lbe = LabelEncoder()
    data[feat] = lbe.fit_transform(data[feat])

In [ ]:
mms = MinMaxScaler(feature_range=(0,1))
data[dense_features] = mms.fit_transform(data[dense_features])

In [ ]:
fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].nunique(),embedding_dim=4)
               for i,feat in enumerate(sparse_features)] + [DenseFeat(feat, 1,)
               for feat in dense_features]

In [ ]:
sparse_feature_columns = [feat for feat in fixlen_feature_columns if isinstance(feat, SparseFeat)]
dense_feature_columns = [feat for feat in fixlen_feature_columns if isinstance(feat, DenseFeat)]


In [ ]:
dnn_feature_columns = sparse_feature_columns + dense_feature_columns
linear_feature_columns = sparse_feature_columns + dense_feature_columns

feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

In [ ]:
from IPython import get_ipython
from IPython.display import display

import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *  # 导入所有模型

if __name__ == "__main__":
    # 读取数据
    data = pd.read_csv('./criteo_sample.txt')

    # 定义稀疏特征和稠密特征
    sparse_features = ['C' + str(i) for i in range(1, 27)]
    dense_features = ['I' + str(i) for i in range(1, 14)]

    # 填充缺失值
    data[sparse_features] = data[sparse_features].fillna('-1', )
    data[dense_features] = data[dense_features].fillna(0, )

    # 定义目标列
    target = ['label']

    # 对稀疏特征进行标签编码，对稠密特征进行归一化
    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
    mms = MinMaxScaler(feature_range=(0, 1))
    data[dense_features] = mms.fit_transform(data[dense_features])

    # 定义特征列
    fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                              for feat in sparse_features] + [DenseFeat(feat, 1, )
                                                              for feat in dense_features]

    # 定义DNN和线性模型的特征列
    dnn_feature_columns = fixlen_feature_columns
    linear_feature_columns = fixlen_feature_columns

    # 获取特征名称
    feature_names = get_feature_names(
        linear_feature_columns + dnn_feature_columns)

    # 划分训练集和测试集
    train, test = train_test_split(data, test_size=0.2)

    # 生成模型输入数据
    train_model_input = {name: train[name] for name in feature_names}
    test_model_input = {name: test[name] for name in feature_names}

    # 设置设备
    device = 'cpu'
    use_cuda = True
    if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

    # 实例化 DeepFM 模型
    model = DeepFM(linear_feature_columns=linear_feature_columns, dnn_feature_columns=dnn_feature_columns,
                   task='binary',
                   l2_reg_embedding=1e-5, device=device)

    # 编译模型
    model.compile("adagrad", "binary_crossentropy",
                  metrics=["binary_crossentropy", "auc"], )

    # 训练模型
    model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2, validation_split=0.0)

    # 预测
    pred_ans = model.predict(test_model_input, 256)
    print("")
    print("测试集 LogLoss", round(log_loss(test[target].values, pred_ans), 4))
    print("测试集 AUC", round(roc_auc_score(test[target].values, pred_ans), 4))

cpu
Train on 160 samples, validate on 0 samples, 5 steps per epoch
Epoch 1/10
0s - loss:  0.6049 - binary_crossentropy:  0.6049 - auc:  0.6198
Epoch 2/10
0s - loss:  0.4926 - binary_crossentropy:  0.4926 - auc:  0.9594
Epoch 3/10
0s - loss:  0.3798 - binary_crossentropy:  0.3798 - auc:  0.9854
Epoch 4/10
0s - loss:  0.2233 - binary_crossentropy:  0.2233 - auc:  0.9981
Epoch 5/10
0s - loss:  0.1210 - binary_crossentropy:  0.1210 - auc:  1.0000
Epoch 6/10
0s - loss:  0.0760 - binary_crossentropy:  0.0760 - auc:  1.0000
Epoch 7/10
0s - loss:  0.0464 - binary_crossentropy:  0.0464 - auc:  1.0000
Epoch 8/10
0s - loss:  0.0326 - binary_crossentropy:  0.0326 - auc:  1.0000
Epoch 9/10
0s - loss:  0.0241 - binary_crossentropy:  0.0241 - auc:  1.0000
Epoch 10/10
0s - loss:  0.0186 - binary_crossentropy:  0.0186 - auc:  1.0000

测试集 LogLoss 0.6906
测试集 AUC 0.5305


In [ ]:
import pandas as pd
import torch
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from deepctr_torch.inputs import SparseFeat, get_feature_names
from deepctr_torch.models import DeepFM


if __name__ == "__main__":
    # 读取数据
    data = pd.read_csv("./movielens_sample.txt")

    # 定义稀疏特征和目标列
    sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
    target = ['rating']

    # 对稀疏特征进行标签编码
    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])

    # 定义特征列
    fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique())
                              for feat in sparse_features]

    # 线性部分和DNN部分共享特征列
    linear_feature_columns = fixlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns

    # 获取特征名称
    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

    # 划分训练集和测试集
    train, test = train_test_split(data, test_size=0.2)

    # 生成模型输入数据
    train_model_input = {name: train[name] for name in feature_names}
    test_model_input = {name: test[name] for name in feature_names}

    # 设置设备
    device = 'cpu'
    use_cuda = True
    if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

    # 实例化 DeepFM 模型
    model = DeepFM(linear_feature_columns, dnn_feature_columns,
                   task='regression', device=device)

    # 编译模型
    model.compile("adam", "mse", metrics=['mse'])

    # 训练模型
    history = model.fit(train_model_input, train[target].values,
              batch_size=256, epochs=10, verbose=2, validation_split=0.2)

    # 预测
    pred_ans = model.predict(test_model_input, batch_size=256)

    # 评估模型
    print("test MSE", round(mean_squared_error(test[target].values, pred_ans), 4))

cpu
Train on 128 samples, validate on 32 samples, 1 steps per epoch
Epoch 1/10
0s - loss:  13.9808 - mse:  13.9808 - val_mse:  12.7248
Epoch 2/10
0s - loss:  13.6224 - mse:  13.6224 - val_mse:  12.4549
Epoch 3/10
0s - loss:  13.3178 - mse:  13.3178 - val_mse:  12.2009
Epoch 4/10
0s - loss:  13.0299 - mse:  13.0299 - val_mse:  11.9613
Epoch 5/10
0s - loss:  12.7569 - mse:  12.7569 - val_mse:  11.7313
Epoch 6/10
0s - loss:  12.4935 - mse:  12.4935 - val_mse:  11.4927
Epoch 7/10
0s - loss:  12.2205 - mse:  12.2205 - val_mse:  11.2453
Epoch 8/10
0s - loss:  11.9378 - mse:  11.9378 - val_mse:  10.9888
Epoch 9/10
0s - loss:  11.6449 - mse:  11.6449 - val_mse:  10.7232
Epoch 10/10
0s - loss:  11.3416 - mse:  11.3416 - val_mse:  10.4484
test MSE 12.3667


In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from torch.nn.utils.rnn import pad_sequence  # 使用 PyTorch 的 pad_sequence

from deepctr_torch.inputs import SparseFeat, VarLenSparseFeat, get_feature_names
from deepctr_torch.models import DeepFM


def split(x):
    """
    将字符串按 '|' 分割，并将每个子字符串编码为数字索引。

    Args:
        x (str): 待分割的字符串。

    Returns:
        list: 编码后的数字索引列表。
    """
    key_ans = x.split('|')
    for key in key_ans:
        if key not in key2index:
            # 输入值 0 用作特殊填充，因此不用 0 编码有效特征
            key2index[key] = len(key2index) + 1
    return list(map(lambda x: key2index[x], key_ans))


if __name__ == "__main__":
    # 读取数据
    data = pd.read_csv("./movielens_sample.txt")
    # 定义稀疏特征和目标列
    sparse_features = ["movie_id", "user_id", "gender", "age", "occupation", "zip"]
    target = ['rating']

    # 1. 对稀疏特征进行标签编码，并处理序列特征
    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])

    # 预处理序列特征
    key2index = {}
    genres_list = list(map(split, data['genres'].values))
    genres_length = np.array(list(map(len, genres_list)))
    max_len = max(genres_length)

    # 使用 pad_sequence 进行填充，注意 padding='post'
    genres_list = pad_sequence([torch.tensor(seq) for seq in genres_list],
                               batch_first=True, padding_value=0)
    genres_list = genres_list.numpy()  # 将填充后的序列转换回 numpy 数组

    # 2. 统计每个稀疏字段的唯一特征数量，并生成序列特征的特征配置
    fixlen_feature_columns = [SparseFeat(feat, data[feat].nunique(), embedding_dim=4)
                              for feat in sparse_features]

    varlen_feature_columns = [VarLenSparseFeat(SparseFeat('genres',
                                  vocabulary_size=len(key2index) + 1,
                                  embedding_dim=4),
                                  maxlen=max_len,
                                  combiner='mean')]
    # 注意：值 0 用于序列输入特征的填充


    linear_feature_columns = fixlen_feature_columns + varlen_feature_columns
    dnn_feature_columns = fixlen_feature_columns + varlen_feature_columns

    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

    # 3. 生成模型输入数据
    model_input = {name: data[name] for name in sparse_features}
    model_input["genres"] = genres_list

    # 4. 定义模型、编译和训练
    device = 'cpu'
    use_cuda = True
    if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

    model = DeepFM(linear_feature_columns, dnn_feature_columns, task='regression', device=device)

    model.compile("adam", "mse", metrics=['mse'])
    history = model.fit(model_input, data[target].values, batch_size=256, epochs=10, verbose=2, validation_split=0.2)

cpu
Train on 160 samples, validate on 40 samples, 1 steps per epoch
Epoch 1/10
0s - loss:  14.1966 - mse:  14.1966 - val_mse:  13.0705
Epoch 2/10
0s - loss:  13.8377 - mse:  13.8377 - val_mse:  12.7593
Epoch 3/10
0s - loss:  13.4977 - mse:  13.4977 - val_mse:  12.4681
Epoch 4/10
0s - loss:  13.1783 - mse:  13.1783 - val_mse:  12.2118
Epoch 5/10
0s - loss:  12.8950 - mse:  12.8950 - val_mse:  11.9611
Epoch 6/10
0s - loss:  12.6174 - mse:  12.6174 - val_mse:  11.7014
Epoch 7/10
0s - loss:  12.3304 - mse:  12.3304 - val_mse:  11.4324
Epoch 8/10
0s - loss:  12.0336 - mse:  12.0336 - val_mse:  11.1540
Epoch 9/10
0s - loss:  11.7269 - mse:  11.7269 - val_mse:  10.8661
Epoch 10/10
0s - loss:  11.4098 - mse:  11.4098 - val_mse:  10.5684


In [ ]:
from IPython import get_ipython
from IPython.display import display

import pandas as pd
import torch
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

from deepctr_torch.inputs import SparseFeat, DenseFeat, get_feature_names
from deepctr_torch.models import *  # 导入所有模型

if __name__ == "__main__":
    # 读取数据
    data = pd.read_csv('./byterec_sample.txt', sep='\t',
                       names=["uid", "user_city", "item_id", "author_id", "item_city", "channel", "finish", "like", "music_id", "device", "time", "duration_time"])

    # 定义稀疏特征和稠密特征
    sparse_features = ["uid", "user_city", "item_id", "author_id", "item_city", "channel", "music_id", "device"]
    dense_features = ["duration_time"]

    # 定义目标列
    target = ['finish', 'like']

    # 1. 对稀疏特征进行标签编码，对稠密特征进行归一化
    for feat in sparse_features:
        lbe = LabelEncoder()
        data[feat] = lbe.fit_transform(data[feat])
    mms = MinMaxScaler(feature_range=(0, 1))
    data[dense_features] = mms.fit_transform(data[dense_features])

    # 2. 统计每个稀疏字段的唯一特征数量，并记录稠密特征字段名称
    fixlen_feature_columns = [SparseFeat(feat, vocabulary_size=data[feat].max() + 1, embedding_dim=4) for feat in sparse_features] \
                   + [DenseFeat(feat, 1, ) for feat in dense_features]

    # 定义DNN和线性模型的特征列
    dnn_feature_columns = fixlen_feature_columns
    linear_feature_columns = fixlen_feature_columns

    # 获取特征名称
    feature_names = get_feature_names(linear_feature_columns + dnn_feature_columns)

    # 3. 生成模型输入数据
    split_boundary = int(data.shape[0] * 0.8)
    train, test = data[:split_boundary], data[split_boundary:]
    train_model_input = {name: train[name] for name in feature_names}
    test_model_input = {name: test[name] for name in feature_names}

    # 4. 定义模型、编译和训练
    device = 'cpu'
    use_cuda = True
    if use_cuda and torch.cuda.is_available():
        print('cuda ready...')
        device = 'cuda:0'

    model = MMOE(dnn_feature_columns, task_types=['binary', 'binary'], l2_reg_embedding=1e-5, task_names=target, device=device)
    model.compile("adagrad", loss=["binary_crossentropy", "binary_crossentropy"], metrics=['binary_crossentropy'], )

    history = model.fit(train_model_input, train[target].values, batch_size=32, epochs=10, verbose=2)
    pred_ans = model.predict(test_model_input, 256)
    print("")
    for i, target_name in enumerate(target):
        print("%s test LogLoss" % target_name, round(log_loss(test[target[i]].values, pred_ans[:, i]), 4))
        print("%s test AUC" % target_name, round(roc_auc_score(test[target[i]].values, pred_ans[:, i]), 4))

cpu
Train on 160 samples, validate on 0 samples, 5 steps per epoch
Epoch 1/10
0s - loss:  1.2010 - binary_crossentropy:  0.3464
Epoch 2/10
0s - loss:  0.8224 - binary_crossentropy:  0.4682


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: T

Epoch 3/10
0s - loss:  0.7603 - binary_crossentropy:  0.4649
Epoch 4/10
0s - loss:  0.7151 - binary_crossentropy:  0.4720
Epoch 5/10
0s - loss:  0.6240 - binary_crossentropy:  0.4508
Epoch 6/10
0s - loss:  0.5370 - binary_crossentropy:  0.4022


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: T

Epoch 7/10
0s - loss:  0.4642 - binary_crossentropy:  0.3761
Epoch 8/10
0s - loss:  0.4122 - binary_crossentropy:  0.3443
Epoch 9/10
0s - loss:  0.3710 - binary_crossentropy:  0.3159
Epoch 10/10
0s - loss:  0.3381 - binary_crossentropy:  0.2989

finish test LogLoss 0.8588
finish test AUC 0.5303
like test LogLoss 0.3686
like test AUC 0.6842


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: The y_pred values do not sum to one. Make sure to pass probabilities.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:3001: UserWarning: T